In [1]:
import sys
import numpy as np
sys.path.append('../')
from model import data_preparation as prep

In [2]:
con = prep.create_connection()
prepared_data = prep.prepare_data(con)

In [3]:
prepared_data

{302: {'abahn': True,
  'brand': 'shell',
  'bstr': True,
  'county': '01051',
  'sstr': True,
  'state': 'schleswig-holstein',
  'time_series': {'test': array([1339, 1329, 1329, ..., 1329, 1489, 1489]),
   'train': array([1399, 1399, 1399, ..., 1339, 1339, 1339])}},
 727: {'abahn': True,
  'brand': 'tankcenter',
  'bstr': True,
  'county': '01056',
  'sstr': False,
  'state': 'schleswig-holstein',
  'time_series': {'test': array([1329, 1329, 1329, ..., 1299, 1389, 1389]),
   'train': array([1329, 1329, 1329, ..., 1329, 1329, 1329])}},
 1885: {'abahn': True,
  'brand': 'esso',
  'bstr': True,
  'county': '03361',
  'sstr': False,
  'state': 'niedersachsen',
  'time_series': {'test': array([1309, 1299, 1299, ..., 1309, 1369, 1429]),
   'train': array([1389, 1389, 1389, ..., 1359, 1319, 1309])}},
 2222: {'abahn': False,
  'brand': 'felta',
  'bstr': True,
  'county': '03251',
  'sstr': False,
  'state': 'niedersachsen',
  'time_series': {'test': array([1314, 1304, 1304, ..., 1304, 1304, 

In [4]:
for key in prepared_data:
    prepared_data[key]["time_series"]["test"] = np.log(prepared_data[key]["time_series"]["test"].reshape(-1,1))
    prepared_data[key]["time_series"]["train"] = np.log(prepared_data[key]["time_series"]["train"].reshape(-1,1))
    

In [6]:
from statsmodels.tsa.stattools import pacf
x_pacf=pacf( prepared_data[2916]['time_series']['train'] , nlags =10, method='ols')

In [7]:
x_pacf

array([ 1.        ,  0.95633229, -0.03137276, -0.02696067, -0.02561561,
       -0.02870637, -0.02368026, -0.01004432,  0.28164104,  0.12105917,
        0.04124528])

In [8]:
from nnet_ts import *
count = 0
ahead =12
pred =[]

Using TensorFlow backend.


ImportError: No module named 'TimeSeriesNnet'